# Предсказание цены автомобиля

Необходимо обучить модель для определения рыночной стоимости автомобиля. В нашем распоряжении следующие данные: технические характеристики, комплектации и цены автомобилей.

## Оглавление:
* [1. Подготовка данных](#1)
* [2. Обучение моделей](#2)
* [3. Анализ моделей](#3)

# 1. Подготовка данных <a class="anchor" id="1"></a>

Импортируем библиотеки:

In [1]:
# <импорт библиотеки pandas>
import pandas as pd

# <импорт библиотеки sklearn>
import sklearn

# <Отключение предупреждений>
import warnings
warnings.filterwarnings('ignore')

# <импорт библиотеки numpy>
import numpy as np

Прочитаем файл с данными:

In [2]:
# <чтение файла с данными с сохранением в переменную df>
df = pd.read_csv('datasets/autos.csv')

Рассмотрим информацию по датафрейму и первые 5 строк:

In [3]:
# <рассмотрим датафрейм df_region_0>
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Подробнее опишем значение каждого атрибута.

Признаки

* DateCrawled — дата скачивания анкеты из базы
* VehicleType — тип автомобильного кузова
* RegistrationYear — год регистрации автомобиля
* Gearbox — тип коробки передач
* Power — мощность (л. с.)
* Model — модель автомобиля
* Kilometer — пробег (км)
* RegistrationMonth — месяц регистрации автомобиля
* FuelType — тип топлива
* Brand — марка автомобиля
* NotRepaired — была машина в ремонте или нет
* DateCreated — дата создания анкеты
* NumberOfPictures — количество фотографий автомобиля
* PostalCode — почтовый индекс владельца анкеты (пользователя)
* LastSeen — дата последней активности пользователя

Целевой признак

* Price — цена (евро)

Создадим отдельный датафрейм, который будем очищать от пропусков.

In [4]:
df_preprocessed=df

Сбросим столбцы дат, они не пригодятся для обучения.

In [5]:
# <сбросим три столбца, которые не помогут нам предсказать целевой признак>
df_preprocessed = df_preprocessed.drop(['DateCrawled','DateCreated','LastSeen'], axis=1)

Рассмотрим столбец *Power*:

In [6]:
df_preprocessed['Power'].describe()

count    354369.000000
mean        110.094337
std         189.850405
min           0.000000
25%          69.000000
50%         105.000000
75%         143.000000
max       20000.000000
Name: Power, dtype: float64

Определим сколько значений столбца *Power* аномальны.

In [7]:
print(df_preprocessed['Power'].quantile(0.11))
print(df_preprocessed['Power'].quantile(0.115))
print(df_preprocessed['Power'].quantile(0.12))

0.0
25.0
43.0


Заменим эти значения на медиану.

In [8]:
power_median = df_preprocessed[df_preprocessed['Power']>25]['Power'].median()

In [9]:
df_preprocessed['Power'] = (np.where((df_preprocessed.Power <= 25), 
                               power_median, 
                               df_preprocessed.Power))

Заполним пропуски в столбце *Model*, опираться будем на бренд и лошадиные силы. Заполним модой - наиболее часто встречающимся значением для определенных группы бренда и кол-ва лошадних сил. Если значения модели нет, будем проставлено значение unknown.

In [10]:
df_preprocessed['Model'] = (df_preprocessed.groupby(['Brand','Power'])['Model'].transform(lambda x: x.fillna((x.mode()[0] if not x.mode().empty else "unknown"))))

Заполним пропуски в столбце *VehicleType*, опираться будем на бренд и модель. Тут мы можем рассчитывать только на заполнение пропусков по моде, так как по имеющимся признакам предположить какой у машины тип кузова мы не в силах.

In [11]:
df_preprocessed['VehicleType'] = (df_preprocessed.groupby(['Brand','Model'])['VehicleType'].transform(lambda x: x.fillna((x.mode()[0] if not x.mode().empty else "unknown"))))

Заполним пропуски в столбце *FuelType*, опираться будем на бренд и модель. Стиль заполнения пропусков тот же.

In [12]:
df_preprocessed['FuelType'] = (df_preprocessed.groupby(['Brand','Model'])['FuelType'].transform(lambda x: x.fillna((x.mode()[0] if not x.mode().empty else "unknown"))))

Заполним пропуски в столбце *Gearbox*, опираться будем на бренд и модель. Стиль заполнения пропусков тот же.

In [13]:
df_preprocessed['Gearbox'] = (df_preprocessed.groupby(['Brand','Model'])['Gearbox'].transform(lambda x: x.fillna((x.mode()[0] if not x.mode().empty else "unknown"))))

Проверим гипотезу:
* Средняя цена у автомобилей с признаком *NotRepaired* = *yes* и *NotRepaired* = *nan* различается.

Сформулируем гипотезы:

**H₀:** Средняя цена у автомобилей с признаком *NotRepaired* = *yes* и *NotRepaired* = *nan* равна.

**H₁:** Средняя цена у автомобилей с признаком *NotRepaired* = *yes* и *NotRepaired* = *nan* различается.

In [15]:
df_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Price              354369 non-null  int64  
 1   VehicleType        354369 non-null  object 
 2   RegistrationYear   354369 non-null  int64  
 3   Gearbox            354369 non-null  object 
 4   Power              354369 non-null  float64
 5   Model              354369 non-null  object 
 6   Kilometer          354369 non-null  int64  
 7   RegistrationMonth  354369 non-null  int64  
 8   FuelType           354369 non-null  object 
 9   Brand              354369 non-null  object 
 10  Repaired           283215 non-null  object 
 11  NumberOfPictures   354369 non-null  int64  
 12  PostalCode         354369 non-null  int64  
dtypes: float64(1), int64(6), object(6)
memory usage: 35.1+ MB


In [16]:
# <Посчитаем среднее для обоих выборок>
yes_mean = df_preprocessed[df_preprocessed['Repaired'] == 'yes']['Price'].mean()
nan_mean = df_preprocessed[(df_preprocessed['Repaired'] != 'no') & (df_preprocessed['Repaired'] != 'yes')]['Price'].mean()
print('Средняя цена отремонтированного автомобиля', yes_mean)
print('Средняя ценам автомобиля, данные по ремонту которого неизвестны:', nan_mean)

Средняя цена отремонтированного автомобиля 1916.0390802684863
Средняя ценам автомобиля, данные по ремонту которого неизвестны: 2626.4192455800094


In [18]:
# <Создадим переменные для наших выборок>
yes_data = df_preprocessed[df_preprocessed['Repaired'] == 'yes']['Price']
nan_data = df_preprocessed[(df_preprocessed['Repaired'] != 'no') & (df_preprocessed['Repaired'] != 'yes')]['Price']

In [19]:
# <импорт библиотеки scipy>
from scipy import stats as st

In [20]:
# <Уровень значимости>
alpha = 0.05

# <Метод библиотеки scipy, позволяющий проверить гипотезу о равенстве двух средних>
results =  st.ttest_ind(
    yes_data, 
    nan_data)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

p-значение: 6.76283705089867e-266
Отвергаем нулевую гипотезу


Такая же ситуация ждет нас и с категорией *no*. Поэтому предлагаю заменить все *nan* значения на третью категорию *unknown*.

In [22]:
df_preprocessed['Repaired'] = (np.where(((df_preprocessed['Repaired'] != 'no') & (df_preprocessed['Repaired'] != 'yes')), 
                               'unknown', 
                               df_preprocessed.Repaired))

Рассмотрим столбец *RegistrationYear*:

In [23]:
df_preprocessed['RegistrationYear'].describe()

count    354369.000000
mean       2004.234448
std          90.227958
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: RegistrationYear, dtype: float64

На сайте *auto.ru* наиболее старый год для поиска объявления - 1890. Сделаем тут также, заменим все аномалии модой.

In [24]:
df_preprocessed['RegistrationYear'] = (np.where(((df_preprocessed['RegistrationYear'] < 1890) | (df_preprocessed['RegistrationYear'] > 2020)), 
                               df_preprocessed['RegistrationYear'].mode(), 
                               df_preprocessed.RegistrationYear))

Рассмотрим столбец *Kilometer*:

In [25]:
df_preprocessed['Kilometer'].describe()

count    354369.000000
mean     128211.172535
std       37905.341530
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: Kilometer, dtype: float64

В нем все довольно адекватно.

Рассмотрим столбец *RegistrationMonth*:

In [26]:
df_preprocessed['RegistrationMonth'].value_counts()

0     37352
3     34373
6     31508
4     29270
5     29153
7     27213
10    26099
12    24289
11    24186
9     23813
1     23219
8     22627
2     21267
Name: RegistrationMonth, dtype: int64

Тут у нас 13 месяцев, но мы можем условиться, что 0 месяц - неизвестный, незаполненный. В условии задачи все равно не было сказано с какого числа считаются месяцы. Также заметим что скорее всего этот показатель будет иметь маленький вес.

Рассмотрим столбец *NumberOfPictures*:

In [27]:
df_preprocessed['NumberOfPictures'].value_counts()

0    354369
Name: NumberOfPictures, dtype: int64

Удалим этот столбец, он никак не будет влиять на обучение, если данные по нему у каждой строки одинаковые. Скорее всего такие значения результат ошибки загрзуки данных или хранения данных. Также удалим столбец *PostalCode*, почтовый индекс клиента анкеты никак не влияет на цену автомобиля.

In [28]:
# <сбросим столбец>
df_preprocessed = df_preprocessed.drop(['NumberOfPictures','PostalCode'], axis=1)

Проверим количество дубликатов:

In [29]:
# <проверим количество полных дубликатов>
df_preprocessed.duplicated().sum()

30187

In [30]:
df_preprocessed.duplicated().sum()/354369

0.08518521653982149

Это плата за мой способ заполнения пропусков. Мы потеряем около 6 % данных.

In [31]:
# <Сбросим дубликаты>
df_preprocessed = df_preprocessed.drop_duplicates().reset_index(drop = True)

### Вывод

Мы осмотрели данные и обработали пропуски. Данные практически подготовлены к обучению.

# 2. Обучение моделей <a class="anchor" id="2"></a>

Переведем тип данных категориальных столбцов в *category*:

In [33]:
df_preprocessed['VehicleType'] = df_preprocessed['VehicleType'].astype('category')
df_preprocessed['Gearbox'] = df_preprocessed['Gearbox'].astype('category')
df_preprocessed['Model'] = df_preprocessed['Model'].astype('category')
df_preprocessed['FuelType'] = df_preprocessed['FuelType'].astype('category')
df_preprocessed['Brand'] = df_preprocessed['Brand'].astype('category')
df_preprocessed['Repaired'] = df_preprocessed['Repaired'].astype('category')

Разделим датафрейм на признаки и целевой признак.

In [34]:
# <Разделим датафрейм на features и target - целевой признак>
target = df_preprocessed['Price']
features = df_preprocessed.drop('Price', axis=1)

Поделим датафрейм на обучающую и тестовую выборку.

In [35]:
# <Импортируем функцию из бибилиотеки sklearn>
from sklearn.model_selection import train_test_split

In [36]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.2, random_state = 0)

Проведем масштабирование признаков:

In [37]:
# <импортируем StandardScaler из библиотеки sklearn>
from sklearn.preprocessing import StandardScaler

In [38]:
numeric = ['RegistrationYear','Power','Kilometer','RegistrationMonth']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

Импортируем библиотеку *LightGBM*:

In [40]:
import lightgbm as lgb

Попробуем использовать эту библиотеку, начнем с дерева принятия решений с градиентным бустингом (*gbdt: traditional Gradient Boosting Decision Tree*).

Загружаем датасет в переменную, создаем словарь с параметрами:
* *boosting_type* - алгоритм градиентного бустинга. Именно тут мы выбрали дерево принятия решений с градиентным бустингом.
* *objective* - тут указывается регрессия это, либо классфикация.
* *metric* - метрика по которой будем оценивать качество модели.
* *learning_rate* - скорость обучения, при ее уменьшении можно получить более лучшие результаты.
* *num_iterations* - количество итераций обучения.
* *max_depth* - максимальная глубина дерева.
* *num_leaves*  - количество листьев дерева.

Время выполнения ячейки - 6 минут 8 секунд.

In [41]:
%%time
hyper_params = {
    'learning_rate': 0.003,
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    "num_iterations": 2500,
    'max_depth': 10,
    'num_leaves': 100
}
gbdt = lgb.LGBMRegressor(**hyper_params)
gbdt.fit(features_train, target_train,
        eval_set=[(features_train, target_train)],
        eval_metric='rmse',
        early_stopping_rounds=10, verbose = 100)

[100]	training's rmse: 3654.05
[200]	training's rmse: 3052.06
[300]	training's rmse: 2636.94
[400]	training's rmse: 2354.31
[500]	training's rmse: 2159.8
[600]	training's rmse: 2027.8
[700]	training's rmse: 1938.38
[800]	training's rmse: 1876.38
[900]	training's rmse: 1833.02
[1000]	training's rmse: 1801.6
[1100]	training's rmse: 1776.95
[1200]	training's rmse: 1758.41
[1300]	training's rmse: 1743.5
[1400]	training's rmse: 1731.42
[1500]	training's rmse: 1721.43
[1600]	training's rmse: 1712.84
[1700]	training's rmse: 1705.1
[1800]	training's rmse: 1698.21
[1900]	training's rmse: 1691.94
[2000]	training's rmse: 1686.12
[2100]	training's rmse: 1680.35
[2200]	training's rmse: 1674.84
[2300]	training's rmse: 1669.7
[2400]	training's rmse: 1665.1
[2500]	training's rmse: 1660.76
CPU times: user 6min 32s, sys: 19.1 s, total: 6min 51s
Wall time: 28.9 s


LGBMRegressor(learning_rate=0.003, max_depth=10, metric='rmse',
              num_iterations=2500, num_leaves=100, objective='regression',
              task='train')

Теперь попробуем случайный лес. С ним обязательно нужно заполнить параметры *feature_fraction*, *bagging_fraction*, *bagging_freq*. Также увеличил количество листьев и кол-во деревьев.

Время выполнения ячейки - 5 минут 22 секунды.

In [42]:
%%time
hyper_params = {
    'task': 'train',
    'boosting_type': 'rf',
    'objective': 'regression',
    'metric': 'rmse',
    "num_iterations": 2500,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.3,
    'bagging_freq': 10,
    'num_leaves': 300,
    'max_depth': 8,
    'n_estimators': 200
    
}
rf = lgb.LGBMRegressor(**hyper_params)
rf.fit(features_train, target_train,
        eval_set=[(features_train, target_train)],
        eval_metric='rmse',
        early_stopping_rounds=10, verbose = 250)

[250]	training's rmse: 2201.53
[500]	training's rmse: 2223.55
[750]	training's rmse: 2221
[1000]	training's rmse: 2218.55
[1250]	training's rmse: 2226
[1500]	training's rmse: 2229.89
[1750]	training's rmse: 2233.26
[2000]	training's rmse: 2232.57
[2250]	training's rmse: 2230.57
[2500]	training's rmse: 2231.56
CPU times: user 6min 29s, sys: 13.1 s, total: 6min 42s
Wall time: 27.2 s


LGBMRegressor(bagging_fraction=0.3, bagging_freq=10, boosting_type='rf',
              feature_fraction=0.5, max_depth=8, metric='rmse',
              n_estimators=200, num_iterations=2500, num_leaves=300,
              objective='regression', task='train')

Теперь попробуем *Dropouts meet Multiple Additive Regression Trees*. Мною были изменены параметры скорости обучения и значения, которое определяет какое количество признаков использовать. Это необходимо чтобы обучение прошло быстрее, так как эта модель обучается медленно.

Время выполнения ячейки - 7 минут 28 секунд.

In [43]:
%%time
hyper_params = {
    'learning_rate': 0.7,
    'task': 'train',
    'boosting_type': 'dart',
    'objective': 'regression',
    'metric': 'rmse',
    "num_iterations": 600,
    'feature_fraction': 0.5
}
dart_model = lgb.LGBMRegressor(**hyper_params)
dart_model.fit(features_train, target_train,
        eval_set=[(features_train, target_train)],
        eval_metric='rmse',
        early_stopping_rounds=10, verbose = 50)

[50]	training's rmse: 1780.17
[100]	training's rmse: 1709.9
[150]	training's rmse: 1668.08
[200]	training's rmse: 1637.68
[250]	training's rmse: 1614.94
[300]	training's rmse: 1598.97
[350]	training's rmse: 1585.44
[400]	training's rmse: 1568.57
[450]	training's rmse: 1552.17
[500]	training's rmse: 1538.35
[550]	training's rmse: 1528.39
[600]	training's rmse: 1525.29
CPU times: user 5min 30s, sys: 9.22 s, total: 5min 39s
Wall time: 23.3 s


LGBMRegressor(boosting_type='dart', feature_fraction=0.5, learning_rate=0.7,
              metric='rmse', num_iterations=600, objective='regression',
              task='train')

Попробуем обучить обыкновенную линейную регрессию.

In [54]:
# <Преобразуем категориальные признаки в фиктивные переменные, и сбросим по одному из них у каждого признака.>
features_train_1 = pd.get_dummies(features_train, drop_first=True)

In [55]:
# <Импортируем метод логистической регрессии>
from sklearn.linear_model import LinearRegression
# <Импортируем функцию cross_val_score>
from sklearn.model_selection import cross_val_score

Время выполнения ячейки - 1 минута 50 секунд.

In [56]:
%%time
# <Создадим модель лог. регрессии,>
model_lr = LinearRegression()

# <Оценим качество модели, обученной в ходе перекрестной проверки>
score = cross_val_score(model_lr, features_train_1, target_train, cv=4, scoring='neg_mean_squared_error').mean()

# <обучаем модель> 
model_lr.fit(features_train_1, target_train)

CPU times: user 2min 22s, sys: 17.3 s, total: 2min 39s
Wall time: 35.7 s


LinearRegression()

### Вывод

Мы обучили несколько моделей с градиентным бустингом, а также линейную регрессию. В следующем пункте проанализируем их скорость работы и качество.

# 3. Анализ моделей <a class="anchor" id="3"></a>

Проверим на тестовой выборке дерево принятия решений с градиентным бустингом.

In [57]:
from sklearn.metrics import mean_squared_error

In [58]:
predictions = gbdt.predict(features_test)
mean_squared_error(target_test,predictions)**0.5

1747.5307562552662

Проверим на тестовой выборке случайный лес с градиентным бустингом.

In [59]:
predictions = rf.predict(features_test)
mean_squared_error(target_test,predictions)**0.5

2115.5786532159013

Проверим на тестовой выборке *Dropouts meet Multiple Additive Regression Trees(DART)*.

In [60]:
predictions = dart_model.predict(features_test)
mean_squared_error(target_test,predictions)**0.5

1721.9203611738253

Проверим на тестовой выборке линейную регрессию.

In [61]:
# <Преобразуем категориальные признаки в фиктивные переменные, и сбросим по одному из них у каждого признака.>
features_test_1 = pd.get_dummies(features_test, drop_first=True)

In [62]:
predictions = model_lr.predict(features_test_1)
mean_squared_error(target_test,predictions)**0.5

3036.4057981081005

Подытожим результаты:

In [63]:
results = {'model': ['Линейная регрессия', 'gbdt','rf','dart'], 
           'rmse': [3036.4, 1747.74, 2114.2, 1726.99],
           'time': ['1:50','6:08','5:22','7:28']
          }
results = pd.DataFrame(data=results)
results

,model,rmse,time
0,Линейная регрессия,3036.40,1:50
1,gbdt,1747.74,6:08
2,rf,2114.20,5:22
3,dart,1726.99,7:28


### Вывод

* Наиболее быстрой в плане обучения является модель линейной регресии. Однако если выставить определенное количество итераций три других метода за то же время что и линейная регрессия дадут лучший результат.
* Среди моделей с градиентным бустингом наиболее быстро обучилась модель случайного леса, но при этом на большинстве итераций не происходило положительной динамики улучшения *RMSE*
* Самой медленной моделью, но при этом выдающей самый качественный результат является модель *DART*, при определенных параметрах и большом количестве итераций можно было получить *RMSE* < 1000.
* Стабильной и в плане скорости, и в плане улучшения *RMSE* показала себя модель *gbdt*.